In [1]:
import os

In [2]:
%pwd

'c:\\Users\\youne\\OneDrive\\Desktop\\End-to-End-Cardiovascular-Disease\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
from Cardiovascular_Disease.constants import *
from Cardiovascular_Disease.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [7]:
import os
from Cardiovascular_Disease import logger
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import joblib

In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        
        

    def label_encode(self):
        label_encoders = {}
        data = pd.read_csv(self.config.data_path)
        for col in ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']:
                le = LabelEncoder()
                data[col] = le.fit_transform(data[col])
                label_encoders[col] = le
        logger.info("Label encoding completed.")
        joblib.dump(label_encoders, 'label_encoders.pkl')
        return data  
 
    
    def scale(self,data):
        scalers = {}
        mms = MinMaxScaler() # Normalization
        data['Oldpeak'] = mms.fit_transform(data[['Oldpeak']])
        scalers['Oldpeak'] = mms

        ss_age = StandardScaler()
        data['Age'] = ss_age.fit_transform(data[['Age']])
        scalers['Age'] = ss_age  

        ss_restingbp = StandardScaler()
        data['RestingBP'] = ss_restingbp.fit_transform(data[['RestingBP']])
        scalers['RestingBP'] = ss_restingbp  

        ss_cholesterol = StandardScaler()
        data['Cholesterol'] = ss_cholesterol.fit_transform(data[['Cholesterol']])
        scalers['Cholesterol'] = ss_cholesterol  

        ss_maxhr = StandardScaler()
        data['MaxHR'] = ss_maxhr.fit_transform(data[['MaxHR']])
        scalers['MaxHR'] = ss_maxhr  

        logger.info("Scaling completed.")

        
        joblib.dump(scalers, 'scalers.pkl')
        return data 


    def feature_engineering(self,data):
         features = data[data.columns.drop(['HeartDisease','RestingBP','RestingECG'])].values
         target = data['HeartDisease'].values
         



    def train_test_spliting(self,data):
        

        
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    encoded_data = data_transformation.label_encode()
    scaled_data = data_transformation.scale(encoded_data)
    data_transformation.train_test_spliting(scaled_data)
except Exception as e:
    raise e

[2024-10-30 20:24:43,060: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-30 20:24:43,304: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-30 20:24:43,309: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-30 20:24:43,347: INFO: common: created directory at: artifacts]
[2024-10-30 20:24:43,356: INFO: common: created directory at: artifacts/data_transformation]
[2024-10-30 20:24:43,771: INFO: 436521754: Label encoding completed.]
[2024-10-30 20:24:43,851: INFO: 436521754: Scaling completed.]
[2024-10-30 20:24:44,149: INFO: 436521754: Splited data into training and test sets]
[2024-10-30 20:24:44,151: INFO: 436521754: (688, 12)]
[2024-10-30 20:24:44,152: INFO: 436521754: (230, 12)]
(688, 12)
(230, 12)
